# **T-SQL Fundamentals**

# **Le Proposizioni**

## Il Flusso Logico di una QUERY

1. **FROM**
2. **WHERE**
3. **GROUP BY**
4. **HAVING**
5. **SELECT**
    - Expressions, Ex. **WINDOW FUNCTIONS**
    - **DISTINCT**
6. **ORDER BY**
    - **TOP/OFFSET-FETCH**

## La proposizione **HAVING.**

Dove la proposizione **WHERE** agisce da filtro per le righe, **HAVING** è il filtro che agisce sui gruppi, accompagnando la proposizione **GROUP BY.**

In [ ]:
SELECT 
    empid,
    YEAR(orderdate) AS orderyear,
    COUNT(*) AS numorders
FROM TSQLV4.Sales.Orders
WHERE custid = 71
GROUP BY empid, YEAR(orderdate)
HAVING COUNT(*) > 1;

## Le proposizioni **SELECT** e **DISTINCT.**

Con la preposizione **SELECT** si elencano i campi (o "colonne") che la query dovrà fornire come risultato, *con o senza manipolazioni*. Inoltre:

- Opzionalmente è possibile specificare la proposizione **AS** per specificare un alias per il campo desiderato. As esempio: ***SELECT Orderdate AS "Data Ordini"***

- Con la proposizione **DISTINCT** E' possibile ottenere il risultato di una query privo di elementi duplicati.


In [ ]:
SELECT DISTINCT 
    empid,
    YEAR(orderdate) AS "Anno Ordine"
FROM TSQLV4.Sales.Orders
WHERE custid = 71;

## La proposizione **ORDER BY.**

La proposizione **ORDER BY** consente di specificare una logica di ordinamento per i risultati prodotti dalla proposizione **SELECT.** In **ORDER BY** è possibile specificare campi non presenti in **SELECT**, come, ad esempio:

In [ ]:
SELECT 
    empid,
    firstname,
    lastname,
    country
FROM TSQLV4.HR.Employees
ORDER BY hiredate;

## **TOP** e **OFFSET-FETCH.**


### **TOP**

Il filtro **TOP** è un costrutto proprietario di T-SQL che è possibile usare per limitare il numero o la percentuale di righe come risultato di una query:

In [ ]:
SELECT TOP (5) 
    orderid,
    orderdate,
    custid,
    empid
FROM TSQLV4.Sales.Orders
ORDER BY orderdate DESC;

In [ ]:
SELECT TOP (1) PERCENT 
    orderid, 
    orderdate,
    custid,
    empid
FROM TSQLV4.Sales.Orders
ORDER BY orderdate DESC;

L'output di una query con il costrutto **TOP** è non-deterministico, cioè, qualora vi siano duplicati o righe di pari valore, non vi è alcuna regola che determini quale partiolare delle *n* righe che rispondono ai requisiti stabiliti verrà scelta. **Vi sono due rimedi:**

- Aggiungere una regola **ORDER BY** *DESC*, in modo tale che la riga con il valore più grande venga preferita alle altre;

- Aggiungere il costrutto **WITH TIES** in modo tale che la query consegni come risultato anche le righe di pari valore;

In [ ]:
SELECT TOP (5) WITH TIES 
    orderid, 
    orderdate, 
    custid, 
    empid
FROM TSQLV4.Sales.Orders
ORDER BY orderdate DESC;

### **OFFSET-FETCH**

Considerato un'estensione di **ORDER BY**, **OFFSET-FETCH** consente di saltare righe (OFFSET) e filtrarne altre (FETCH). Il codice seguente salterà le prime 50 righe e mostrerà le seguenti 10:


In [ ]:
SELECT orderid, orderdate, custid, empid
FROM TSQLV4.Sales.Orders
ORDER BY orderdate, orderid
OFFSET 50 ROWS FETCH NEXT 10 ROWS ONLY;

Tramite **OFFSET** 0 **ROWS** è possibile evitare di saltare righe, indicando in **FETCH FIRST** *N* **ROWS ONLY** il numero di righe da mostrare. In tal modo il codice si comporterà come **SELECT TOP (N)**.

In [ ]:
SELECT orderid, orderdate, custid, empid
FROM TSQLV4.Sales.Orders
ORDER BY orderdate, orderid
OFFSET 0 ROWS FETCH FIRST 10 ROWS ONLY;

## **CASE-WHEN-THEN** Expressions.

Un **CASE** è un'espressione scalare, cioè restituisce un unico risultato, la quale produce un valore in base ad un'espressione logica condizionale. Esempi:

In [ ]:
SELECT productid, productname, categoryid,
    CASE categoryid
        WHEN 1 THEN 'Beverages'
        WHEN 2 THEN 'Condiments'
        WHEN 3 THEN 'Confections'
        WHEN 4 THEN 'Dairy'
        WHEN 5 THEN 'Grains'
        WHEN 6 THEN 'Meat'
        WHEN 7 THEN 'Produce'
        WHEN 8 THEN 'Seafood'
        ELSE 'Unknown'
    END AS categoryname
FROM TSQLV4.Production.Products

In [ ]:
SELECT orderid, custid, val,
    CASE 
        WHEN val < 1000 THEN 'Less than 1000'
        WHEN val BETWEEN 1000.00 AND 3000.00 THEN 'Between 1000 and 3000'
        WHEN val > 3000 THEN 'More than 3000'
        ELSE 'Unknown'
    END AS valuecategory
FROM TSQLV4.Sales.OrderValues

## **NULLs.**

T-SQL adotta una logica a tre valori. Da tale condizione risultano:

- Un'espressione logica fra valori non nulli può risultare solo in **TRUE** o **FALSE**. 

- Quando un'espressione logica coinvolge un **NULL**, allora darà come risultato **UNKNOWN**. **UNKNOWN** negato risulta sempre in **UNKNOWN.**

- Curiosamente quando un'espressione coinvolge due **NULL** risulterà in **UNKNOWN.**

- **GROUP BY** e **ORDER BY** considerano **NULL** differenti come lo stesso valore: verranno dunque raggruppati o ordinati come una singola classe.

Per tutte le query di ricerca è dunque essenziale prevedere una casistica per i risultati **NULL.**

# **Esplorare i metadati di un database**

## @SQL SERVER — **.SYS** Catalog Views

Le **Catalog Views** sono funzioni di SQL Server che permettono di ottenere informazioni dettagliate circa gli oggetti contenuti in un database, come ad esempio le liste di schemas e tabelle.

### > Esplorare Schemas e Tabelle di un Database con **sys.tables**

In [ ]:
USE TSQLV4;
SELECT SCHEMA_NAME(schema_id) AS table_schema_name, name AS table_name
FROM sys.tables;

### > Esplorare le colonne di una tabella con **sys.columns**

In [ ]:
--Elenca tutte le colonne della tabella Sales.Orders e le loro proprietà.
SELECT
    name AS 'Colonna',
    TYPE_NAME(system_type_id) AS 'Tipo Dati',
    max_length AS 'Lunghezza Massima',
    collation_name AS 'Collation',
    is_nullable AS 'Accetta NULL?'
FROM sys.columns
WHERE object_id = OBJECT_ID('Sales.Orders');

-----------------------------------------------------------------------------------------------------------------------------------------------

--La stessa query con SELECT * ritorna una quantità molto esaustiva di informazioni.

SELECT * 
FROM sys.columns
WHERE object_id = OBJECT_ID('Sales.Orders');

## @SQL STANDARD — **INFORMATION_SCHEMA** Views

Funzionalità standard di SQL — la quale, dunque, non prevede metadati od oggetti specifici a SQL Server — che risiede nello schema **INFORMATION_SCHEMA**.

In [ ]:
--QUERY STANDARD per elencare Schema e Tabelle presenti nel database.
USE TSQLV4;
SELECT TABLE_SCHEMA, TABLE_NAME
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_TYPE = N'BASE TABLE';

----------------------------------------------------------------------------------------------------------------------

--El
SELECT COLUMN_NAME, DATA_TYPE, CHARACTER_MAXIMUM_LENGTH, COLLATION_NAME, IS_NULLABLE
FROM INFORMATION_SCHEMA.COLUMNS
WHERE (TABLE_SCHEMA = N'Sales') AND (TABLE_NAME = N'Orders')

## **Stored Procedures** di Sistema

Le **stored procedures** (capitolo dedicato più avanti) sono delle query SQL pre-compilate che vengono memorizzate nel database per il loro riutilizzo, una cui efficace implementazione riduce considerevolmente il carico di lavoro per i fruitori della database, siano essi gli amministratori o gli utenti finali, rendendo dispobile una libreria di codice efficiente e riutilizzabile. SQL Server dispone di una serie di **stored procedures per l'estrazione di metadati dal sistema:**

- **sys.sp_tables** — produce una lista di tabelle e view che possono essere ricercati nel database;

In [ ]:
USE TSQLV4;
EXEC sys.sp_tables;

- **sys.sp_help** — fornisce informazioni generiche su un oggetto, come: colonne, tipi di dati, collation, indici, constraints e altro;

In [ ]:
USE TSQLV4;
EXEC sys.sp_help @objname = N'Sales.Orders'

- **sys.sp_columns** — fornisce informazioni sulle colonne di un oggetto;

In [ ]:
USE TSQLV4;
EXEC sys.sp_columns @table_name = N'Orders', @table_owner = N'Sales';

- **sys.sp_helpconstraint** — fornisce informazioni sui **constraint** presenti in un oggetto;

In [ ]:
USE TSQLV4;
EXEC sys.sp_helpconstraint @objname = N'Sales.Orders';

## Funzioni per l'Estrazione di Metadati

Oltre a Views e Stored Procedures, vi sono anche funzioni utili per l'estrazione di metadati:

- **DATABASEPROPERTYEX(database_name, property)** — restituisce la proprietà *property* del *database_name* specificato

In [ ]:
SELECT DATABASEPROPERTYEX(N'TSQLV4', 'Collation');

- **OBJECTPROPERTY(object_id, property)** — restituisce la proprietà *property* dell'oggetto definito dall'*object_id* indicato

In [ ]:
SELECT OBJECTPROPERTY(OBJECT_ID(N'Sales.Orders'), 'TableHasPrimaryKey');

# **Lavorare con Stringhe e Caratteri**

### **Tipi di Caratteri**

SQL Server support due tipi di caratteri: 

- **Regolari** — tipi **CHAR(n)** e **VARCHAR(n-MAX)**, consigliati per supportare una sola lingua oltre all'inglese.

    - *'Questa è una stringa di caratteri regolari'*
- **Unicode** — tipi **NCHAR(n)** e **NVARCHAR(n-MAX)**, consigliati per supportare molteplici lingue.

    - *N'Questa è una stringa Unicode'*



### **_CHAR e _VARCHAR**

A seguire alcune peculiarità tecniche dei principali campi stringa disponibili:

- **CHAR** e **NCHAR** rappresentano stringhe di un numero fisso, prestabilito di caratteri. Ad esempio, **CHAR(5)** identifica una stringa di massimo 5 caratteri, come ***'ciao'***. Un'ipotetico input ***'Cicciomerda'*** in un campo identificato da un tipo **CHAR(5)** verrebbe automaticamente troncato a ***'Ciccio'***, cioè la sua rappresentazione a 5 caratteri. Per ragioni tecniche, **la dimensione fissa di CHAR rende più veloci le operazioni su questi campi vs. VARCHAR**.

- **VARCHAR** e **NVARCHAR** invece rappresentano campi di stringhe di lunghezza variabile. Per tale ragione è consigliabile inizializzare tali campi come **_VARCHAR(MAX)**. Per ragioni tecniche, la lettura sui campi **_VARCHAR** è più performante rispetto ai campi **_CHAR**.



### **Collation**

La **Collation** è una proprietà dei dati di tipo stringa che racchiude diverse caratteristiche, come:

- Supporto Linguistico 
- Ordine di ordinamento
- Case sensitivity

In una comune installazione *on-premise* di SQL, la **Collation** può essere definita a quattro diversi livelli:

- **Istanza** — La collation di un'istanza è definita al momento dell'installazione e viene in tale sede stabilita come il valore di **DEFAULT** per tutti i database utente e di sistema.
- **Database** — Specificata mediante la chiave **COLLATE**, oppure stabilita con il valore di default. Stabilisce la collation per le tabelle ed i metadati (e.g. *Oggetti* e *Campi*)dello specifico database.
- **Colonna** —  Stabilita mediante **COLLATE** oppure assunta come il default del database.
- **Espressione** — Stabilita mediante **COLLATE** oppure assunta come il valore di default.


## **Operatore "+" e CONCAT(string1, string2, ..., string*n*)**

L'operatore **+** consente la concatenazione di strighe. Secondo SQL standard, la concatenazione di una qualsiasi stringa con NULL da esito NULL. Tale evenienza è rimediata dalla funzione **CONCAT( )**, la quale sostituisce ai valori NULL delle stringhe vuote ' '.

In [ ]:
SET NOCOUNT ON --Riduce i messaggi a completamento delle query.

SELECT empid, 
       firstname + N' ' + lastname AS 'Full Name'
FROM TSQLV4.HR.Employees;

SELECT TOP (10) custid, country, region, city, country + N', ' + region +N', ' + city AS 'Location'
FROM TSQLV4.Sales.Customers

SELECT TOP (10) custid, country, region, city, CONCAT(country, N', ' + region, N', ' + city) AS 'Location'
FROM TSQLV4.Sales.Customers;


## **LEN(string) — DATALENGTH(string)**

Producono la lunghezza in caratteri o la lunghezza in bytes di una stringa. <strong>LEN(string)</strong> ignora gli spazi in coda, mentre <strong>DATALENGTH(string)</strong> li considera.

In [ ]:
SELECT LEN('Cacca') AS 'Character Length',
       DATALENGTH('Cacca') AS 'Ascii Datalength', 
       DATALENGTH(N'Cacca') AS 'Unicode Datalength';

## **SUBSTRING(string, start, length) — LEFT(string, n) — RIGHT(string, n)**

Opera sulla stringa *string* per selezionare la sottostringa di lunghezza *length* a partire dalla posizione *start*. Se la lunghezza *length* supera la sottostringa disponibile dopo *start*, **SUBSTRING()** restituirà la sottostringa fino all'ultimo carattere. Sostituendo *length* con **LEN(string)** possiamo ottenere dinamicamente l'intera sottostringa a partire dal carattere in posizione *start*.

In [ ]:
SELECT SUBSTRING('cacca', 2, 4) AS "SUBSTRING()",
       SUBSTRING('cacca', 2, LEN('cacca')) AS "SUBSTRING() + LEN()",
       LEFT('Cacca', 3) AS "LEFT('Cacca', 3')",
       RIGHT('Cacca', 3) AS "RIGHT('Cacca', 3')";

## **CHARINDEX(substring, string, [start]) — PATINDEX(pattern, string)**

La funzione **CHARINDEX** restituisce la posizione di *substring* all'interno dell'argomento *string*, dando la possibilità di specificare dove far partire la ricerca con l'argomento *start*. La funzione **PATINDEX** invece restituisce la posizione di un *pattern* all'interno di una *string*.

In [ ]:
SELECT CHARINDEX('pupù', 'Caccapupù') AS "CHARINDEX('pupù', 'Caccapupù')",
       PATINDEX('%[0-9]%', 'Ginopastino.90') AS "PATINDEX('%[0-9]%', 'Ginopastino.90')";

## **REPLACE(string, substring1, substring2) — REPLICATE(string, n)**

La funzione **REPLACE** ha il compito di sostituire tutte le occorrenze di *substring1* con *substring2* all'interno dell'argomento *string*. La funzione **REPLICATE**, invece, replicherà l'argomento *string* per *n* volte.

In [ ]:
SELECT REPLACE(REPLICATE('a', 20), 'a', 'b');

### > Contare il numero di occorrenze di un carattere in una stringa con **REPLACE**:

In [ ]:
--Uso REPLACE() per sostituire il carattere cercato con una stringa vuota.
--Misuro la differenza fa la stringa risultante e quella originale.
SELECT empid, lastname, LEN(lastname) - LEN(REPLACE(lastname, 'e','')) AS 'Occurrences'
FROM TSQLV4.Hr.Employees

## **STUFF(string, position, length, insert)**

La funzione **STUFF** opera in *string* sostituendo rimuovendo *length* caratteri alla posizione *position* e vi inserisce una stringa *insert*:

In [ ]:
--Imito il meccanismo di REPLACE con la funzione STUFF
SELECT STUFF('Caccamerda', CHARINDEX('merda', 'Caccamerda'), LEN('merda'), 'pupù') AS 'STUFF()',
       REPLACE('Caccamerda', 'merda', 'pupù') AS 'REPLACE()';

## **UPPER/LOWER(string) e RTRIM/LTRIM(string)**

Consentono la trasformazione di *string* da ad Uppercase o Lowercase. Mentre le funzioni **RTRIM/LTRIM** consentono l'eliminazione di spazi in testa o in coda a *string*

In [ ]:
SELECT '  Cacca  ' AS 'Regular', 
       UPPER('  Cacca  ') AS 'Uppercase', 
       LOWER('  Cacca  ') AS 'Lowercase',
       RTRIM(LTRIM('  Cacca  ')) AS 'No Leading/Trailing Spaces'

## **COMPRESS/DECOMPRESS(string/VARBINARY)**

Le funzioni **COMPRESS/DECOMPRESS** utilizzano l'algoritmo GZIP per comprimere o decomprimere l'argomento.

- **COMPRESS** — Accetta una stringa di caratteri o codice binario come input e restituisce un valore **VARBINARY(MAX)**

- **DECOMPRESS** — Accetta un **VARBINARY(MAX)** in ingresso e lo consegna decompresso. Se il valore originario era di tipo NCHAR/NVARCHAR(MAX), allora sarà necessario incpsulare DECOMPRESS in un blocco **CAST()**, come segue:
        CAST(DECOMPRESS(value)) AS NVARCHAR(MAX)



## **SELECT *value* FROM STRING_SPLIT(string, separator)**

Al contrario delle funzioni scalari elencate finora, **STRING_SPLIT** è una **funzione tabulare** e cioè una funzione che produce una tabella, anzichè consegnare un singolo valore. Per questa ragione, la sintassi di **STRING_SPLIT** è differente. La funzione ha il compito di suddividere una stringa in elementi individuali, identificati da un carattere separatore — *e.g.* "," - "|" - "\\" etc....

In [ ]:
SELECT CAST(value AS INT) AS 'Valore'
FROM STRING_SPLIT('1111,2222,3333,4444,5555', ',');

## **Il predicato LIKE**

Il predicato **LIKE** si può utilizzare per verificare se una stringa contiene un determinato pattern/combinazione. **LIKE** si può configurare mediante l'uso delle seguenti **wildcard**:

- **%** — Rappresenta una stringa di qualsiasi dimensione, inclusa la stringa vuota.

- **_** — Rappresenta un unico carattere.

- **[lista di caratteri]** — Indica un singolo carattere che deve appartenere alla lista specificata. Ad esempio **[ABC]** 

- **[carattere-carattere]** — Indica un range di caratteri. Ad esempio **[A-E]** indica qualsiasi carattere dalla A alla E, inclusa. 

- **[^lista o range di caratteri]** — La negazione della condizione precedente. Indica un range di caratteri che la stringa **NON** deve contenere. 

- **ESCAPE** — E' utilizzato per verificare una stringa per la presenze di un carattere a sua volta utilizzato nel codice come wildcard. Tuttavia per le wildcard %, _, [, è possibile utilizzare la notazione %[ _ ]%.

In [ ]:
SET NOCOUNT ON

-- Cerca tutti gli impiegati il cui cognome inizi per "D".
SELECT empid, lastname
FROM TSQLV4.HR.Employees
WHERE lastname LIKE N'D%';

-- Cerca tutti gli impiegati il cui cognome abbia come seconda lettera "e".
SELECT empid, lastname
FROM TSQLV4.HR.Employees
WHERE lastname LIKE N'_e%';

-- Cerca tutti gli impiegati il cui cognome inizi per A, B o C. 
SELECT empid, lastname
FROM TSQLV4.HR.Employees
WHERE lastname LIKE N'[ABC]%';

-- Cerca tutti gli impiegati le cui iniziali del cognome siano una lettera compresa fra A ed E.
SELECT empid, lastname
FROM TSQLV4.HR.Employees
WHERE lastname LIKE N'[A-E]%';

-- Cerca tutti gli impiegati le cui iniziali del cognome ESCLUDANO i caratteri dalla A alla E.
SELECT empid, lastname
FROM TSQLV4.HR.Employees
WHERE lastname LIKE N'[^A-E]%';

# **Lavorare con Date e Ore**

## **Tipi di dati**

T-SQL supporta sei tipi di dati data/ora:

- **DATETIME** (Legacy) — YYYYMMDD hh:mm:ss.nnn, 8 bytes. Monolitico: data e tempo sono inseparabili.

- **SMALLDATETIME** (Legacy) — YYYYMMDD hh:mm, 4 bytes. Monolitico.

- **DATETIME2** — YYYYMMDD/YYYY-MM-DD hh:mm:ss.nnnnnnn, 6-8 bytes. Le  componenti DATE e TIME si possono separare. 

- **DATETIMEOFFSET** — YYYYMMDD/YYYY-MM-DD hh:mm:ss.nnnnnnn [+|-] hh:mm. 8-10 bytes. Simile a DATETIME2 ma include il fuso orario rispetto a UTC.

- **DATE** — YYYYMMDD/YYYY-MM-DD. 3 bytes.

- **TIME** — hh:mm:ss.nnnnnnn. 3-5 bytes.

 
## **Codificare un dato DATETIME**

T-SQL non prevede una notazione specifica per codificare un dato DATETIME, bensì si appoggia sulla **conversione, implicita o esplicita, di una stringa**. Ad esempio:

        


In [ ]:
-- Conversione implicita STRING -> DATETIME
SELECT orderid, custid, empid, orderdate
FROM TSQLV4.Sales.Orders
WHERE orderdate = '20160212';

Nel codice qui sopra, T-SQL converte implicitamente la stringa '20160212' ad un dato di tipo DATETIME, poichè la colonna **orderdate** è stata inizializzata con quel tipo di dato. Qualora per chiarezza o robustezza del codice volessimo esplicitare tale conversione, si potrebbe codificare in modo del tutto analogo la seguente query:

In [ ]:
-- Conversione esplicita STRING -> DATETIME
SELECT orderid, custid, empid, orderdate
FROM TSQLV4.Sales.Orders
WHERE orderdate = CAST('20160212' AS DATE);

Infine, per tutti i tipi che includono sia una componente DATE che una componente TIME, la codifica standard è la seguente:

- **Se non viene specificato TIME** — SQL assume mezzanotte, 00:00:00:...

- **Se non viene specificato alcun fuso/offset UTC** — SQL assume 00:00

## **Effetti di Localizzazione e Lingua di Sistema sui dati DATETIME**

Le espressioni con DATETIME sono soggette a differenti interpretazioni a seconda della localizzazione e/o della lingua di sistema definita ad ogni login dell'amministratore del database. Tale impostazione, se non esplicitamente cambiata mediante il comando **SET LANGUAGE**, diventa effettiva per la sessione in corso, andando a determinare il senso particolare in cui saranno interpretate le date, se queste sono **NON LANGUAGE NEUTRAL**, come ad esempio **"02/12/2016"**, che nel codice seguente viene interpretata differentemente a seconda delle impostazioni linguistiche della sessione. Al contrario, la **notazione standard YYYYMMDD/YYYY-MM-DD** sarà considerata tale a prescidere dal setup.

In [ ]:
-- Verrà interpretato come 2 Dicembre, 2016
SET LANGUAGE British
SELECT CAST('02/12/2016' AS DATE);

-- Verrà intepretato come 12 Febbraio, 2016
SET LANGUAGE us_english
SELECT CAST('02/12/2016' AS DATE);

### **Utilizzare Deliberatamente una Notazione Language-Dependent con CONVERT/PARSE e gli Style Numbers**

In [ ]:
--Dichiara 02/12/2016 come MM/DD/YYYY con lo Style Number 101
SELECT CONVERT(DATE, '02/12/2016', 101);

--Dichiara 02/12/2016 come DD/MM/YYYY con lo Style Number 101
SELECT CONVERT(DATE, '02/12/2016', 103);

--ALTO COSTO COMPUTAZIONALE!!! - Dichiara 02/12/2016 come DATE, specificando la cultura en-US
SELECT PARSE('02/12/2016' AS DATE USING 'en-US');

## **UTC Time Zones in T-SQL**

In T-SQL le informazioni sulle zone UTC sono disponibili nella tabella di sistema **sys.time_zone_info**:

In [ ]:
SELECT 
    current_utc_offset AS 'UTC Offset', 
    name AS 'Description', 
    CASE is_currently_dst 
        WHEN 1 THEN 'YES' 
        ELSE 'NO' 
    END AS 'Daylight Saving Right Now?' 
FROM sys.time_zone_info
ORDER BY current_utc_offset ASC;

## Filtrare Range di Date con **YEAR/MONTH/DAY(datetime)** e ottimizzare le query per l'indicizzazone



In [ ]:
SET NOCOUNT ON

-- Cerco tutti l'ultimo 10% degli ordini piazzati nell'anno 2015, dal più recente al meno recente.
SELECT TOP (10) PERCENT orderid, custid, empid, orderdate
FROM TSQLV4.Sales.Orders
WHERE YEAR(orderdate) = 2015
ORDER BY orderid DESC;

Tuttavia, usare funzioni per filtrare/manipolare una colonna impedisce a SQL di massimizzare le sue prestazioni utilizzando appropriatamente gli indici. Alla luce di ciò, la query precedente può essere riscritta come segue:

In [ ]:
--Query precedente ottimizzata per l'indicizzazione. L'esecuzione è circa 2ms più rapida.
SELECT TOP (10) orderid, custid, empid, orderdate
FROM TSQLV4.Sales.Orders
WHERE (orderdate >= '20150101') AND (orderdate < '20160101')
ORDER BY orderid DESC;

---------------------------------------------------------------------------------------------------------------------------------

--Query con FUNZIONE, NON ottimizzata per l'indicizzazione, per cercare gli ultimi 10 ordini piazzati a Febbraio 2016 
SELECT TOP (10) orderid, custid, empid, orderdate
FROM TSQLV4.Sales.Orders
WHERE (YEAR(orderdate) = 2016) AND (MONTH(orderdate) = 2)
ORDER BY orderid DESC;

--Query con FILTRO, ottimizzata per l'indicizzazione, per cercare tutti gli ultimi 10 ordini piazzati a Febbraio 2016
SELECT TOP (10) orderid, custid, empid, orderdate
FROM TSQLV4.Sales.Orders
WHERE (orderdate >= '20160201') AND (orderdate < '20160301')
ORDER BY orderid DESC;

## Ottenere **DATE** e **TIME** correnti con **SYSDATETIME()**, **SYSUTCDATETIME()** e **SYSDATETIMEOFFSET()** e altre funzioni


In [ ]:
--FUNZIONI CONSIGLIATE
SELECT
    SYSDATETIME() AS 'SYSDATETIME()', --Ritorna DATETIME2, Data e Ora correnti
    SYSUTCDATETIME() AS 'SYSUTCDATETIME', --Ritorna DATETIME2, Data e Ora correnti al fuso UTC
    SYSDATETIMEOFFSET() AS 'SYSDATETIMEOFFSET()'; --Ritorna DATETIMEOFFSET, Data e Ora correnti + Fuso Orario rispetto a UTC
    
---------------------------------------------------------------------------------------------------

--LEGACY FUNCTIONS
SELECT
    GETDATE() AS 'GETDATE()', --Ritorna DATETIME (LEGACY)
    CURRENT_TIMESTAMP AS 'CURRENT_TIMESTAMP', --Ritorna DATETIME (LEGACY) ma ANSI-standard
    GETUTCDATE() AS 'GETUTCDATE()'; --Ritorna DATETIME (LEGACY) e fuso UTC

## Convertire le ore con **SWITCHOFFSET(datetimeoffset, UTC_offset)**

La funzione **SWITCHOFFSET** di convertire un valore DATETIMEOFFSET ad uno specifico fuso orario UTC 'hh:mm' :

In [ ]:
--Converte la data/ora di sistema al fuso orario +08:00
SELECT SWITCHOFFSET(SYSDATETIMEOFFSET(), '+08:00');

--Converte la data/ora di sistema al fuso UTC
SELECT SWITCHOFFSET(SYSDATETIMEOFFSET(), '+00:00');

## Unire DATETIME e UTC da fonti legacy con **TODATETIMEOFFSET(local_datetime_value, UTC_offset)**

Usata in particolare **per migrare dati non-UTC_offset a dati UTC_offset**, TODATETIMEOFFSET unisce (e non converte!) un dato DATETIME/DATETIME2 con un UTC_offset specificato per creare un nuovo dato di tipo DATETIMEOFFSET.

In [ ]:
SELECT TODATETIMEOFFSET(SYSDATETIME(), '+08:00');

## Addizionare date con **DATEADD(datetime_part, n, datetime_value)**

La funzione **DATEADD** consente di aggiungere ***n*** *year, quarter, month, day, week, hour, minute, second, millisecond, microsecond, nanosecond* specificato/i nell'argomento *datetime_part* ad uno specifico valore dato nell'argomento *datetime_value*.

In [ ]:
--Aggiunge un anno alla data odierna
SELECT DATEADD(year, 1, SYSDATETIMEOFFSET());

## Differenza fra date con **DATEDIFF/DATEDIFF_BIG(datetime_part, datetime1, datetime2)**

Le funzioni **DATEDIFF** e **DATEDIFF_BIG** consentono di calcolare la differenza in year, quarter, month, day, week, hour, minute, second, millisecond, microsecond, nanosecond fra un valore *datetime1* ed un altro *datetime2*. 

L'unica differenza fra le due è la grandezza dell'intero resistuito: **INT** se composto di 4 byte o **BIGINT**, se composto da 8 byte. Quest'ultimo pensato per calcoli più grandi come confronti fra piccole unità temporali come millisecondi/microsecondi e via dicendo.

In [ ]:
--Calcola il numero di giorni trascorsi dall'inizio dell'anno 2020
SELECT DATEDIFF(day, '20200101', SYSDATETIME()) AS "Giorno dell'anno";

--Calcola il numero di microsecondi trascorsi dall'inizio dell'anno 2020
SELECT DATEDIFF_BIG(microsecond, '20200101', SYSDATETIME()) AS "Microsecondi dall'inizio dell'anno";

## Estrarrei il nome di una data con **DATENAME(part, datetime)**



In [ ]:
SELECT DATENAME(month, '20200101');

## Produrre DATETIME da parti separate: le funzioni **...FROMPARTS(year, month, day, ...)**

In [ ]:
SELECT DATEFROMPARTS(2020, 08, 02);
SELECT DATETIME2FROMPARTS(2020, 08, 02, 23, 00, 00, 00, 00);